# Smartnoise Synth

In [1]:
from lomas_client import Client

In [2]:
APP_URL = "http://lomas_server"
USER_NAME = "Dr. Antartica"
DATASET_NAME = "BIRTHDAYS"
client = Client(url=APP_URL, user_name = USER_NAME, dataset_name = DATASET_NAME)

In [3]:
import datetime
import random
import uuid
import numpy as np
import pandas as pd

pums_csv_path = "https://raw.githubusercontent.com/opendp/smartnoise-sdk/main/datasets/PUMS.csv"
pums = pd.read_csv(pums_csv_path, index_col=None) # in datasets/

uuids = [str(uuid.uuid4()) for _ in range(len(pums))]
pums['guid'] = uuids

uuids2 = [str(uuid.uuid4()).replace('-', '') for _ in range(len(pums))]
pums['guid2'] = uuids2

dates = [datetime.datetime.now() + random.randint(0, 1000) * datetime.timedelta(days=1) for _ in range(len(pums))]
dates = [d.isoformat() for d in dates]
pums['when'] = dates

def rand_char():
    return chr(random.randint(ord('a'), ord('z')))
def rand_digit():
    return random.randint(0, 9)
def make_email():
    return f"{rand_char()}{rand_char()}{rand_char()}{rand_digit()}@{rand_char()}{rand_char()}{rand_char()}.com"
emails = [make_email() for _ in range(len(pums))]
pums['email'] = emails

def make_ssn():
    return f"{rand_digit()}{rand_digit()}{rand_digit()}-{rand_digit()}{rand_digit()}-{rand_digit()}{rand_digit()}{rand_digit()}{rand_digit()}"
ssns = [make_ssn() for _ in range(len(pums))]
pums['ssn'] = ssns

In [4]:
pums

,age,sex,educ,race,income,married,guid,guid2,when,email,ssn
0,59,1,9,1,0.0,1,61f75aa8-5234-42ca-9502-a8e95b6b1d7f,eb7ed3f9d4984f9ebc5b29dde1725ce1,2024-11-25T14:05:03.010005,iwq5@abh.com,716-49-5993
1,31,0,1,3,17000.0,0,f09aca52-87c9-43dc-b715-042de276213f,27de8c302e8541cb9b54950bb9887bf0,2027-05-03T14:05:03.010040,uza7@jnn.com,427-42-3060
2,36,1,11,1,0.0,1,7403a937-7de5-4025-85b2-49239ab2f347,5ae3702fe7444d98bb2ab2609f770683,2027-01-01T14:05:03.010050,aom6@gzp.com,418-44-4818
3,54,1,11,1,9100.0,1,06377376-5603-4d72-880f-486ebf3b526d,367df8aea36b48eebee5612459b2a910,2025-06-04T14:05:03.010056,uzi1@vsc.com,836-19-4431
4,39,0,5,3,37000.0,0,406ed9ee-ec36-41e6-ae34-a709697ab664,6be283191062453a8cefde7e834d58af,2026-12-27T14:05:03.010061,vea3@ibt.com,785-97-1108
...,...,...,...,...,...,...,...,...,...,...,...
995,73,0,3,3,24200.0,0,75428dd5-5eee-4f6e-b466-3557b50abc9e,726839ceb52445119977a6f42dc844bc,2026-09-02T14:05:03.015796,pmg8@hyv.com,337-87-0843
996,38,1,2,3,0.0,0,a9cfd518-ccd2-420c-a8a7-0bd5206b69e0,f0b0159178c44f158e1306e99e68f7d6,2025-07-17T14:05:03.015803,anq1@vnl.com,090-21-7332
997,50,0,13,1,22000.0,1,1da46348-21c6-4e04-bfe5-a7ffd688c7eb,e33db662ecf0422f99ed800d07da24b6,2027-01-21T14:05:03.015810,wcd9@jzh.com,081-60-9605
998,44,1,14,4,500.0,1,9daa4bda-5e67-40e3-b1be-5ab9ddfe219d,2db4a35266c84587a6337254a3b3dce5,2025-02-07T14:05:03.015817,wlc7@uwt.com,908-40-5070


In [168]:
max(pums["when"])

'2027-05-17T09:15:56.934173'

In [180]:
epoch = datetime.strptime("2024-08-20", "%Y-%m-%d")
parsed = datetime.strptime("2024-08-20", "%Y-%m-%d")
distance = parsed - epoch
float(distance.total_seconds() / (60 * 60 * 24))

0.0

In [181]:
epoch = datetime.strptime("2024-08-20", "%Y-%m-%d")
parsed = datetime.strptime("2027-05-17", "%Y-%m-%d")
distance = parsed - epoch
float(distance.total_seconds() / (60 * 60 * 24))

1000.0

In [182]:
from datetime import datetime

constraints["when"] = ChainTransformer(
    [
        DateTimeTransformer(),
        MinMaxTransformer(
            lower=0.0,
            upper=10000,
            nullable=True,
        ),
    ]
)

In [183]:
tt = TableTransformer.create(pums, style='cube', constraints=constraints)

In [184]:
tt.fit(pums, epsilon=4.0)

In [185]:
pums_encoded = tt.transform(pums)
pums_decoded = tt.inverse_transform(pums_encoded)

In [186]:
vars(tt)

{'transformers': [<snsynth.transform.label.LabelTransformer at 0x7fe6a79e0910>,
 'output_width': 8,
 'odometer': <snsql.sql.odometer.OdometerHeterogeneous at 0x7fe6a78be450>,
 '_columns': Index(['age', 'sex', 'educ', 'race', 'income', 'married', 'guid', 'guid2',
        'when', 'email', 'ssn'],
       dtype='object'),
 '_dtype': None,
 '_dropped_column_indices': set()}

In [162]:
dt_t = tt.transformers[-3]
dt_t

In [163]:
vars(dt_t)

{'transformers': [<snsynth.transform.datetime.DateTimeTransformer at 0x7fe6a7c32590>,
 '_fit_complete': False,
 'output_width': 1,
 '_fit_vals': []}

In [164]:
dt_tt = dt_t.transformers[0]
vars(dt_tt)

{'_fit_complete': True,
 'output_width': 1,
 'epoch': datetime.datetime(1970, 1, 1, 0, 0),
 'format': 'datetime',
 'string': True}

In [165]:
dt_tt = dt_t.transformers[1]
vars(dt_tt)

{'lower': None,
 'upper': None,
 'epsilon': 2.0,
 'bins': 20,
 'budget_spent': [2.0],
 'nullable': False,
 'odometer': <snsql.sql.odometer.OdometerHeterogeneous at 0x7fe6a7f455d0>,
 '_fit_complete': True,
 'output_width': 1,
 '_fit_vals': [20724.386075573482,
  20472.38607557389,
  20773.386075573995,
  20372.386075574075,
  20385.386075574133,
  20292.386075574203,
  20834.386075574283,
  20535.386075574384,
  20646.38607557449,
  19993.38607557456,
  20281.386075574643,
  20032.386075574712,
  20073.38607557478,
  20464.38607557485,
  20791.38607557492,
  20610.38607557499,
  20147.386075575057,
  20820.386075575127,
  20573.386075575196,
  20713.386075575265,
  19962.386075575334,
  20434.386075575392,
  20147.38607557546,
  20005.38607557553,
  20191.386075575603,
  20488.386075575672,
  20552.38607557574,
  20955.3860755758,
  20299.38607557587,
  20449.386075575938,
  20128.386075575996,
  19955.386075576065,
  20331.386075576134,
  20785.386075576203,
  20235.386075576273,
  208

In [196]:
metadata = client.get_dataset_metadata()
metadata

{'max_ids': 1,
 'row_privacy': True,
 'censor_dims': False,
 'columns': {'birthday': {'type': 'datetime',
   'lower': '1900-01-01',
   'upper': '2050-01-01'}}}

In [197]:
import pandas as pd

In [198]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/dscc-admin-ch/lomas/wip_306_snsynth-datetime/server/data/datasets/birthdays.csv",
    dtype={'birthday': 'string'},
    parse_dates=['birthday']
)
df

,birthday
0,1903-12-11
1,2003-08-13
2,1987-10-30
3,1924-11-30
4,2007-08-01


In [199]:
col = 'birthday'

In [211]:
def datetime_to_float(upper, lower):
    distance = datetime.strptime(upper, "%Y-%m-%d") - datetime.strptime(lower, "%Y-%m-%d")
    return float(distance.total_seconds() / (60 * 60 * 24))

In [212]:
datetime_to_float(metadata["columns"][col]["upper"], metadata["columns"][col]["lower"])

54787.0

In [220]:
from snsynth.transform import ChainTransformer, MinMaxTransformer
from snsynth.transform.datetime import DateTimeTransformer

constraints = {}
col = "birthday"


constraints[col] = ChainTransformer(
    [
        DateTimeTransformer(epoch = metadata["columns"][col]["lower"]),
        BinTransformer(
            lower=0.0,
            upper=datetime_to_float(metadata["columns"][col]["upper"], metadata["columns"][col]["lower"]),
            bins=20,
            nullable=True,
        ),
    ]
)

#constraints[col] = DateTimeTransformer()

constraints[col] = ChainTransformer(
    [
        DateTimeTransformer(epoch = metadata["columns"][col]["lower"]),
        MinMaxTransformer(
            lower=0.0,
            upper=datetime_to_float(metadata["columns"][col]["upper"], metadata["columns"][col]["lower"]),
            nullable=True,
        ),
    ]
)

In [221]:
from snsynth.transform.table import TableTransformer

transformer = TableTransformer.create(
    data=df,
    style="gan",
    nullable=True,
    constraints=constraints,
)

In [222]:
from snsynth import Synthesizer

model = Synthesizer.create(
    synth="dpctgan",
    epsilon=2.0,
    verbose=True,
    batch_size=4,
)

In [223]:
model.fit(
    data=df,
    transformer=transformer,
    preprocessor_eps=0.0,  # will error if not 0.0
    nullable=True,
)

Epoch 1, Loss G: 0.7122, Loss D: 1.4228
epsilon is -0.09366297356767976, alpha is 10.9
Epoch 2, Loss G: 0.7281, Loss D: 1.3334


/usr/local/lib/python3.11/site-packages/opacus/privacy_engine.py:638: UserWarning: The sample rate will be defined from ``batch_size`` and ``sample_size``.The returned privacy budget will be incorrect.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/opacus/privacy_engine.py:229: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


epsilon is 0.10060114003648732, alpha is 5.9
Epoch 3, Loss G: 0.7639, Loss D: 1.4554
epsilon is 0.24056837218518043, alpha is 4.9
Epoch 4, Loss G: 0.6920, Loss D: 1.2705
epsilon is 0.3619566639546349, alpha is 4.4
Epoch 5, Loss G: 0.7009, Loss D: 1.3476
epsilon is 0.4724823145466076, alpha is 4.1
Epoch 6, Loss G: 0.7456, Loss D: 1.4077
epsilon is 0.5754943495669649, alpha is 3.8
Epoch 7, Loss G: 0.7213, Loss D: 1.3811
epsilon is 0.672992493310269, alpha is 3.6
Epoch 8, Loss G: 0.7431, Loss D: 1.4923
epsilon is 0.7657749981485822, alpha is 3.5
Epoch 9, Loss G: 0.7420, Loss D: 1.4427
epsilon is 0.8552081335918694, alpha is 3.4
Epoch 10, Loss G: 0.6859, Loss D: 1.4550
epsilon is 0.9415916593503026, alpha is 3.3
Epoch 11, Loss G: 0.7231, Loss D: 1.4370
epsilon is 1.0252823473193244, alpha is 3.2
Epoch 12, Loss G: 0.6882, Loss D: 1.5559
epsilon is 1.1067082712716174, alpha is 3.1
Epoch 13, Loss G: 0.6512, Loss D: 1.3468
epsilon is 1.1863875417467915, alpha is 3.0
Epoch 14, Loss G: 0.7233, L

In [224]:
model.epsilon_list[-1], model.delta

(2.039661658526861, 0.08944271909999159)

In [225]:
model.sample(100)

,birthday
0,2032-03-20 21:33:19.624157
1,1981-02-03 11:57:30.337565
2,2026-12-03 18:32:20.586376
3,1991-10-01 01:33:10.943241
4,2016-07-13 18:44:08.988247
...,...
95,2018-07-28 04:04:38.103447
96,2015-03-09 03:30:56.837082
97,2025-09-30 06:21:20.976677
98,2009-11-07 21:02:46.767311


### DPCTGAN

In [230]:
cost = client.estimate_smartnoise_synth_cost(
    synth_name="dpctgan",
    epsilon= 1.0,
    delta = 0.0001,
    select_cols = [],
    synth_params = {"batch_size": 4},
    nullable = True,
    table_transformer_style = "gan",
    constraints = "",
)
cost

{'epsilon_cost': 1.1819369953217957, 'delta_cost': 0.0001}

In [238]:
res_dummy = client.smartnoise_synth_query(
    synth_name="dpctgan",
    epsilon=1.0,
    delta=0.0001,
    select_cols = [],
    synth_params = {
        "embedding_dim": 128,
        "generator_dim": (256, 256),
        "discriminator_dim": (256, 256),
        "batch_size": 2,
        "epochs": 5
    },
    nullable = True,
    table_transformer_style = "gan",
    dummy = True,
    constraints = "",
    return_model=True,
    nb_samples=10,
    condition="",
    nb_rows=3,
    seed=0
)
res_dummy

/usr/local/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))
/usr/local/lib/python3.11/site-p

{'query_response': <snsynth.pytorch.nn.dpctgan.DPCTGAN at 0x7fe6a7879a50>}

In [240]:
model = res_dummy["query_response"]
model.sample(5)

,birthday
0,1972-04-18 22:21:11.450061
1,1988-10-21 21:03:47.441454
2,1966-08-29 11:24:57.194910
3,1950-12-05 05:44:35.309944
4,2001-01-05 07:45:41.369104


In [1]:
res = client.smartnoise_synth_query(
    synth_name="dpctgan",
    epsilon=1.0,
    delta=0.0001,
    select_cols = [],
    synth_params = {
        "embedding_dim": 128,
        "generator_dim": (256, 256),
        "discriminator_dim": (256, 256),
        "batch_size": 2,
        "epochs": 5
    },
    nullable = True,
    table_transformer_style = "gan",
    dummy = False,
    constraints = "",
    return_model=True,
    nb_samples=10,
    condition=""
)
res

NameError: name 'client' is not defined

In [235]:
model = res["query_response"]

In [236]:
model.sample(5)

,birthday
0,1988-03-28 08:24:43.095932
1,1984-02-22 17:33:05.342252
2,1960-10-13 11:47:09.323673
3,2013-04-08 14:02:44.365768
4,1963-10-01 07:17:57.141237


In [41]:
model.sample(5).columns

Index(['age', 'sex', 'educ', 'race', 'income', 'married'], dtype='object')

In [ ]:
dataset_name='PUMS' 
synth_name=<SSynthSynthesizer.DP_CTGAN: 'dpctgan'> 
epsilon=1.0 
delta=1e-05 
select_cols=[] 
synth_params={'embedding_dim': 128, 'generator_dim': [256, 256], 'discriminator_dim': [256, 256], 'batch_size': 2, 'epochs': 5} 
nullable=True 
table_transformer_style=<SSynthTableTransStyle.GAN: 'gan'> 
constraints='' 
dummy_nb_rows=100 
dummy_seed=42

In [ ]:
dataset_name='PUMS', 
synth_name='dpctgan', 
epsilon=0.1, 
delta=1e-05, 
select_cols=[], 
synth_params={'embedding_dim': 128, 'generator_dim': (256, 256), 'discriminator_dim': (256, 256), 'batch_size': 50, 'epochs': 5}, 
nullable=True, 
table_transformer_style='gan', 
constraints='', 
dummy_nb_rows=100, 
dummy_seed=42

In [31]:
QUERY_EPSILON = 0.1
QUERY_DELTA = 0.00001
TABLE_TRANSFORMER_STYLE = "gan"

example_smartnoise_synth = {
    "dataset_name": "PENGUIN",
    "synth_name": "dpctgan",
    "epsilon": QUERY_EPSILON,
    "delta": QUERY_DELTA,
    "select_cols": [],
    "synth_params": {
        "embedding_dim": 128,
        "generator_dim": (256, 256),
        "discriminator_dim": (256, 256),
        "batch_size": 50,
        "epochs": 5,
    },
    "nullable": True,
    "table_transformer_style": TABLE_TRANSFORMER_STYLE,
    "constraints": "",
}

In [33]:
example_dummy_smartnoise_synth = dict(example_smartnoise_synth)
example_dummy_smartnoise_synth["dummy_nb_rows"] = 100
example_dummy_smartnoise_synth["dummy_seed"] = 42

In [34]:
body = dict(example_dummy_smartnoise_synth)
body["dataset_name"] = "PUMS"
body

{'dataset_name': 'PUMS',
 'synth_name': 'dpctgan',
 'epsilon': 0.1,
 'delta': 1e-05,
 'select_cols': [],
 'synth_params': {'embedding_dim': 128,
  'generator_dim': (256, 256),
  'discriminator_dim': (256, 256),
  'batch_size': 50,
  'epochs': 5},
 'nullable': True,
 'table_transformer_style': 'gan',
 'constraints': '',
 'dummy_nb_rows': 100,
 'dummy_seed': 42}

In [25]:
!pip install pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 7.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [35]:
from pydantic import BaseModel, Field
from typing import List, Optional

class SmartnoiseSynthModel(BaseModel):
    """Model input for a smarnoise-synth query"""

    dataset_name: str
    synth_name: str
    epsilon: float = Field(..., gt=0, le=10)
    delta: Optional[float] = None
    select_cols: List
    synth_params: dict
    nullable: bool
    table_transformer_style: str
    constraints: str

class DummySmartnoiseSynthModel(SmartnoiseSynthModel):
    """Dummy Model input for a smarnoise-synth query"""

    dummy_nb_rows: int = Field(..., gt=0)
    dummy_seed: int

In [36]:
query_json_pums = DummySmartnoiseSynthModel.model_validate(body)

In [37]:
query_json_pums

DummySmartnoiseSynthModel(dataset_name='PUMS', synth_name='dpctgan', epsilon=0.1, delta=1e-05, select_cols=[], synth_params={'embedding_dim': 128, 'generator_dim': (256, 256), 'discriminator_dim': (256, 256), 'batch_size': 50, 'epochs': 5}, nullable=True, table_transformer_style='gan', constraints='', dummy_nb_rows=100, dummy_seed=42)

In [11]:
model.__class__.__name__

'DPCTGAN'

In [60]:
vars(model)

{'_embedding_dim': 128,
 '_generator_dim': (256, 256),
 '_discriminator_dim': (256, 256),
 '_generator_lr': 0.0002,
 '_generator_decay': 1e-06,
 '_discriminator_lr': 0.0002,
 '_discriminator_decay': 1e-06,
 '_batch_size': 2,
 '_discriminator_steps': 1,
 '_verbose': True,
 '_epochs': 5,
 'pac': 1,
 'sigma': 5,
 'disabled_dp': False,
 'delta': 0.001,
 'max_per_sample_grad_norm': 1.0,
 'epsilon': 1.0,
 'epsilon_list': [0.028590312727600813,
  0.08218579056833558,
  0.12402234203412246,
  0.15744994423302555,
  0.18636981922339252],
 'alpha_list': [63.0, 57.0, 43.0, 37.0, 33.0],
 'loss_d_list': [tensor(1.3969, requires_grad=True),
  tensor(1.4062, requires_grad=True),
  tensor(1.3785, requires_grad=True),
  tensor(1.3978, requires_grad=True),
  tensor(1.4244, requires_grad=True)],
 'loss_g_list': [tensor(0.6773, requires_grad=True),
  tensor(0.6624, requires_grad=True),
  tensor(0.6593, requires_grad=True),
  tensor(0.6754, requires_grad=True),
  tensor(0.6652, requires_grad=True)],
 'verb

In [32]:
import pandas as pd

In [38]:
df = pd.read_csv("https://raw.githubusercontent.com/opendp/smartnoise-sdk/main/datasets/PUMS.csv")
df.shape[0]

1000

In [41]:
df.dtypes

age          int64
sex          int64
educ         int64
race         int64
income     float64
married      int64
dtype: object

In [34]:
model = Synthesizer.create(
    synth="mwem",
    epsilon=1.0,
    verbose=True,
)

In [16]:
res = client.smartnoise_synth_query(
    synth_name="mwem",
    epsilon= 2.0,
    #delta = 0.0001,
    select_cols = [],
    synth_params = {},
    nullable = True,
    table_transformer_style = "gan",
    dummy = True,
    constraints = "",
)
res

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

{'query_response': <snsynth.pytorch.nn.dpctgan.DPCTGAN at 0x7f3daa605650>}

In [ ]:
model = res["query_response"]

In [ ]:
model.sample(5)

In [ ]:
model.__class__.__name__

In [ ]:
from snsynth.transform import ChainTransformer, LabelTransformer, MinMaxTransformer, OneHotEncoder
from smartnoise_synth_logger import serialise_constraints

constraints = {
   'species': ChainTransformer([LabelTransformer(nullable=True), OneHotEncoder()]),
   'island': ChainTransformer([LabelTransformer(nullable=True), OneHotEncoder()]),
   'bill_length_mm': MinMaxTransformer(lower=30.0, upper=65.0, nullable=True),
   'bill_depth_mm': MinMaxTransformer(lower=13.0, upper=23.0, nullable=True),
   'flipper_length_mm': MinMaxTransformer(lower=150.0, upper=250.0, nullable=True),
   'body_mass_g': MinMaxTransformer(lower=2000.0, upper=7000.0, nullable=True),
   'sex': ChainTransformer([LabelTransformer(nullable=True), OneHotEncoder()]),
}

In [51]:
c = serialise_constraints(constraints)
c

'{"module": "smartnoise-synth", "version": "1.0.4", "constraints": {"species": {"type": "_ssynth_transformer:ChainTransformer", "params": [{"type": "_ssynth_transformer:LabelTransformer", "params": {"nullable": true}}, {"type": "_ssynth_transformer:OneHotEncoder", "params": {}}]}, "island": {"type": "_ssynth_transformer:ChainTransformer", "params": [{"type": "_ssynth_transformer:LabelTransformer", "params": {"nullable": true}}, {"type": "_ssynth_transformer:OneHotEncoder", "params": {}}]}, "bill_length_mm": {"type": "_ssynth_transformer:MinMaxTransformer", "params": {"lower": 30.0, "upper": 65.0, "epsilon": 0.0, "negative": true, "nullable": true, "odometer": null}}, "bill_depth_mm": {"type": "_ssynth_transformer:MinMaxTransformer", "params": {"lower": 13.0, "upper": 23.0, "epsilon": 0.0, "negative": true, "nullable": true, "odometer": null}}, "flipper_length_mm": {"type": "_ssynth_transformer:MinMaxTransformer", "params": {"lower": 150.0, "upper": 250.0, "epsilon": 0.0, "negative": tr

In [55]:
res = client.smartnoise_synth_query(
    synth_name="dpctgan",
    epsilon= 2.0,
    delta = 0.0001,
    select_cols = [],
    synth_params = {
        "embedding_dim": 128, 
        "generator_dim": (256, 256), 
        "discriminator_dim": (256, 256),
        "batch_size": 2
    },
    nullable = True,
    constraints = constraints,
    table_transformer_style = "gan",
    dummy = True
)
res

/usr/local/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


{'query_response': <snsynth.pytorch.nn.dpctgan.DPCTGAN at 0x7f2e0be779d0>}

In [57]:
model = res["query_response"]
model.sample(2)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,64.979197,13.000297,150.001132,6999.272376,FEMALE
1,Adelie,Torgersen,64.995704,13.000055,150.000179,6999.830872,FEMALE


## Error 1: test_smartnoise_synth_query_transformer_type

In [42]:
res = client.smartnoise_synth_query(
    synth_name="dpctgan",
    epsilon= 2.0,
    delta = 0.0001,
    select_cols = [],
    synth_params = {
        "embedding_dim": 128, 
        "generator_dim": (256, 256), 
        "discriminator_dim": (256, 256),
        "batch_size": 2
    },
    nullable = True,
    table_transformer_style = "cube",
    dummy = True
)
res

/usr/local/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


{'query_response': <snsynth.pytorch.nn.dpctgan.DPCTGAN at 0x7f2e0c4a7450>}

In [43]:
model_full = res["query_response"]

In [44]:
df = model_full.sample(10)

TypeError: list indices must be integers or slices, not numpy.float32

In [32]:
vars(model_full).keys()

dict_keys(['_embedding_dim', '_generator_dim', '_discriminator_dim', '_generator_lr', '_generator_decay', '_discriminator_lr', '_discriminator_decay', '_batch_size', '_discriminator_steps', '_verbose', '_epochs', 'pac', 'sigma', 'disabled_dp', 'delta', 'max_per_sample_grad_norm', 'epsilon', 'epsilon_list', 'alpha_list', 'loss_d_list', 'loss_g_list', 'verbose', 'loss', '_device', '_transformer', '_data_sampler', '_generator'])

In [48]:
#vars(model_full)

In [37]:
model_full.delta

0.0001

In [27]:
df = model_full.sample(5)

In [34]:
list(df.columns) ==['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex']

True

In [16]:
from snsynth.transform.type_map import TypeMap

In [5]:
import pandas as pd
from snsynth import Synthesizer
from snsynth.transform import (
    AnonymizationTransformer,
    BinTransformer,
    ChainTransformer,
    #DateTimeTransformer,
    LabelTransformer,
    MinMaxTransformer,
    OneHotEncoder,
)
from snsynth.transform.table import TableTransformer

from typing import Dict, List, Optional

In [6]:
from snsynth.transform.datetime import DateTimeTransformer

In [7]:
Synthesizer.list_synthesizers()

['mwem', 'dpctgan', 'patectgan', 'mst', 'pacsynth', 'dpgan', 'pategan', 'aim']

In [10]:
!pip install pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 580.4 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 2.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [11]:
from pydantic import BaseModel, Field
class SmartnoiseSynthModel(BaseModel):
    """Model input for a smarnoise-synth query"""

    dataset_name: str
    synth_name: str
    epsilon: float = 0.0
    delta: float = 0.0
    select_cols: List = []
    synth_params: dict = {}
    mul_matrix: List = []
    nullable: bool = True
    condition: Optional[str] = None
    nb_samples: Optional[int] = None
    table_transformer_style: str = "gan"

In [12]:
from enum import StrEnum
class SSynthSynthesizer(StrEnum):
    """Synthesizer models for smartnoise synth"""

    # Marginal Synthesizer
    AIM = "aim"
    MWEM = "mwem"
    MST = "mst"
    PAC_SYNTH = "pacsynth"

    # Neural Network Synthesizer
    DP_CTGAN = "dpctgan"
    PATE_CTGAN = "patectgan"
    PATE_GAN = "pategan"  # no documentation
    DP_GAN = "dpgan"  # no documentation

    # Hybrid Synthesizer
    QUAIL = "quail"


class SSynthTableTransStyle(StrEnum):
    """Transformer style for smartnoise synth"""

    GAN = "gan"
    CUBE = "cube"


class SSynthColumnType(StrEnum):
    """Type of columns for SmartnoiseSynth transformer pre-processing"""

    PRIVATE_ID = "private_id"
    CATEGORICAL = "categorical"
    CONTINUOUS = "continuous"
    ORDINAL = "ordinal"
    DATETIME = "datetime"


SSYNTH_PRIVATE_COLUMN = "uuid"
SSYNTH_DEFAULT_NB_SAMPLES = 100

In [13]:
def _preprocess_data(
        private_data: pd.DataFrame,
        query_json: dict,
    ) -> pd.DataFrame:
        """
        Preprocess the data based on the query parameters.

        Args:
            private_data (pd.DataFrame): Private data to be preprocessed
            query_json (dict): (SmartnoiseSynthModelCost): JSON request object
                select_cols (List[str]): List of columns to select
                mul_matrix (List): Multiplication matrix for columns aggregations

        Returns:
            pd.DataFrame: Preprocessed private data
        """
        if query_json.select_cols:
            try:
                private_data = private_data[query_json.select_cols]
            except KeyError as e:
                raise ValueError(
                    "Error while selecting provided select_cols: " + str(e)
                ) from e

        if query_json.mul_matrix:
            try:
                np_matrix = np.array(query_json.mul_matrix)
                mul_private_data = private_data.to_numpy().dot(np_matrix.T)
                private_data = pd.DataFrame(mul_private_data)
            except ValueError as e:
                raise ValueError(
                    f"Failed to multiply provided mul_matrix: {(str(e))}"
                ) from e
        return private_data

In [14]:
def _categorize_column(data: dict) -> str:
    """
    Categorize the column based on its metadata.

    Args:
        data (dict): Metadata of the column.

    Returns:
        str: Category of the column.
    """
    match data["type"]:
        case "string" | "boolean":
            return SSynthColumnType.CATEGORICAL
        case "int" | "float":
            if "lower" in data.keys():
                return SSynthColumnType.CONTINUOUS
            if "cardinality" in data.keys():
                return SSynthColumnType.CATEGORICAL
            return SSynthColumnType.ORDINAL
        case "datetime":
            return SSynthColumnType.DATETIME
        case _:
            raise ValueError(
                f"Unknown column type in metadata: {data['type']}"
            )

def _get_column_by_types(
    metadata,
    select_cols: List[str],
) -> Dict[str, List[str]]:
    """
    Sort the column in categories based on their types and metadata

    Args:
        metadata (Metadata): Metadata of the dataset
        select_cols (List[str]): List of columns to select

    Returns:
        Dict[str, List[str]]: Dictionnary of list of columns by categories
    """
    col_categories: Dict[str, List[str]] = {
        SSynthColumnType.CATEGORICAL: [],
        SSynthColumnType.CONTINUOUS: [],
        SSynthColumnType.DATETIME: [],
        SSynthColumnType.ORDINAL: [],
        SSynthColumnType.PRIVATE_ID: [],
    }
    for col_name, data in metadata["columns"].items():
        if select_cols and col_name not in select_cols:
            continue
        
        if "private_id" in data.keys():
            col_categories[SSynthColumnType.PRIVATE_ID].append(col_name)
            continue

        # Sort the column in categories based on their types and metadata
        category = _categorize_column(data)
        col_categories[category].append(col_name)

    return col_categories

def _prepare_data_transformer(
    metadata,
    private_data: pd.DataFrame,
    query_json: dict,
):
    """
    Creates the transformer based on the metadata
    The transformer is used to transform the data before synthesis and then
    reverse the transformation after synthesis.
    See https://docs.smartnoise.org/synth/transforms/index.html for documentation
    See https://github.com/opendp/smartnoise-sdk/blob/main/synth/snsynth/
        transform/type_map.py#L40 for get_transformer() method taken as basis.

    Args:
        metadata (Metadata): Metadata of the dataset
        private_data
        query_json (SmartnoiseSynthModelCost): JSON request object for the query
            select_cols (List[str]): List of columns to select
            nullable (bool): True is the data can have Null values, False otherwise
            table_transformer_style: 'gan' or 'cube'

    Returns:
        table_tranformer (TableTransformer) to pre and post-process the data
    """
    col_categories = _get_column_by_types(metadata, query_json.select_cols)
    style = query_json.table_transformer_style
    nullable = query_json.nullable

    constraints = {}
    for col in col_categories[SSynthColumnType.PRIVATE_ID]:
        constraints[col] = AnonymizationTransformer(SSYNTH_PRIVATE_COLUMN)

    if style == SSynthTableTransStyle.GAN:
        for col in col_categories[SSynthColumnType.CATEGORICAL]:
            constraints[col] = ChainTransformer(
                [LabelTransformer(nullable=nullable), OneHotEncoder()]
            )
        for col in col_categories[SSynthColumnType.CONTINUOUS]:
            constraints[col] = MinMaxTransformer(
                lower=metadata["columns"][col]["lower"],
                upper=metadata["columns"][col]["upper"],
                nullable=nullable,
            )
        for col in col_categories[SSynthColumnType.DATETIME]:
            constraints[col] = ChainTransformer(
                [
                    DateTimeTransformer(),
                    MinMaxTransformer(
                        lower=metadata["columns"][col]["lower"],
                        upper=metadata["columns"][col]["upper"],
                        nullable=nullable
                    ),
                ]
            )
        for col in col_categories[SSynthColumnType.ORDINAL]:
            constraints[col] = ChainTransformer(
                [LabelTransformer(nullable=nullable), OneHotEncoder()]
            )
    else:
        for col in col_categories[SSynthColumnType.CATEGORICAL]:
            constraints[col] = LabelTransformer(nullable=nullable)
        for col in col_categories[SSynthColumnType.CONTINUOUS]:
            constraints[col] = BinTransformer(nullable=nullable)
        for col in col_categories[SSynthColumnType.DATETIME]:
            constraints[col] = ChainTransformer(
                [
                    DateTimeTransformer(),
                    BinTransformer(bins=20, nullable=nullable),
                ]
            )
        for col in col_categories[SSynthColumnType.ORDINAL]:
            constraints[col] = LabelTransformer(nullable=nullable)
    print(constraints)
    return TableTransformer.create(
        data=private_data,
        style=style,
        nullable=nullable,
        constraints=constraints,
    )

## Synthesize data

In [15]:
query_json_pums = {
    "dataset_name": "PENGUIN",
    "synth_name": "dpctgan",
    "epsilon": 0.1,
    "delta": 0.00001,
    "synth_params": {},
    "select_cols": [],
    "mul_matrix": [],
    "nullable": True,
    "condition": None,
    "nb_samples": None,
}
query_json_pums = SmartnoiseSynthModel.model_validate(query_json_pums)

### PUMS

In [16]:
pums_private_data = pd.read_csv("https://raw.githubusercontent.com/opendp/smartnoise-sdk/main/datasets/PUMS.csv")
print(pums_private_data.head())
pums_metadata = {
    "columns": {
        "age": {'type': 'int', 'lower': 18, 'upper': 70},
        "sex": {'type': 'boolean'},
        "educ": {'type': 'int', 'cardinality': 14},
        "race": {'type': 'int', 'cardinality': 6},
        "income": {'type': 'float', 'lower': 0.0, 'upper': 500_000},
        "married": {'type': 'boolean'},
    }
}

   age  sex  educ  race   income  married
0   59    1     9     1      0.0        1
1   31    0     1     3  17000.0        0
2   36    1    11     1      0.0        1
3   54    1    11     1   9100.0        1
4   39    0     5     3  37000.0        0


In [17]:
pums_private_data = _preprocess_data(pums_private_data, query_json_pums)

In [18]:
pums_transformer = _prepare_data_transformer(pums_metadata, pums_private_data, query_json_pums)
pums_transformer

{'sex': <snsynth.transform.chain.ChainTransformer object at 0x7f55d12ab350>, 'educ': <snsynth.transform.chain.ChainTransformer object at 0x7f55d145e910>, 'race': <snsynth.transform.chain.ChainTransformer object at 0x7f55d145d050>, 'married': <snsynth.transform.chain.ChainTransformer object at 0x7f55d145d010>, 'age': <snsynth.transform.minmax.MinMaxTransformer object at 0x7f55d145e110>, 'income': <snsynth.transform.minmax.MinMaxTransformer object at 0x7f55d1465950>}


In [19]:
pums_encoded = pums_transformer.fit_transform(pums_private_data, epsilon=0.0)
pums_transformer.odometer.spent

(0.0, 0.0)

In [23]:
model = Synthesizer.create(
    synth="pacsynth",
    epsilon=query_json_pums.epsilon,
    **query_json_pums.synth_params,
)

In [22]:
model.fit(
    data=pums_private_data,
    transformer=pums_transformer, #pums_transformer,
    preprocessor_eps=0.0, #0.0,  # will error if not 0.
    #nullable=query_json_pums.nullable,
)

ValueError: The transformer appears to have some continuous columns. Please provide only categorical or ordinal.

In [ ]:
model

In [ ]:
samples = model.sample(5)
print(samples)

### Penguins

In [24]:
penguin_private_data = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv", index_col=None)
print(penguin_private_data.head())
penguin_metadata = {
 'max_ids': 1,
 'row_privacy': True,
 'censor_dims': False,
 'columns': {
   'species': {'type': 'string', 'cardinality': 3,'categories': ['Adelie', 'Chinstrap', 'Gentoo']},
   'island': {'type': 'string', 'cardinality': 3, 'categories': ['Torgersen', 'Biscoe', 'Dream']},
   'bill_length_mm': {'type': 'float', 'lower': 30.0, 'upper': 65.0},
   'bill_depth_mm': {'type': 'float', 'lower': 13.0, 'upper': 23.0},
   'flipper_length_mm': {'type': 'float', 'lower': 150.0, 'upper': 250.0},
   'body_mass_g': {'type': 'float', 'lower': 2000.0, 'upper': 7000.0},
   'sex': {'type': 'string', 'cardinality': 2,'categories': ['MALE', 'FEMALE']}
  }
}

  species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7              181.0   
1  Adelie  Torgersen            39.5           17.4              186.0   
2  Adelie  Torgersen            40.3           18.0              195.0   
3  Adelie  Torgersen             NaN            NaN                NaN   
4  Adelie  Torgersen            36.7           19.3              193.0   

   body_mass_g     sex  
0       3750.0    MALE  
1       3800.0  FEMALE  
2       3250.0  FEMALE  
3          NaN     NaN  
4       3450.0  FEMALE  


In [38]:
query_json_penguin = {
    "dataset_name": "PENGUIN",
    "synth_name": "pacsynth",
    "epsilon": 2.0,
    "delta": 0.00001,
    "select_cols": [],
    "mul_matrix": [],
    "nullable": True,
    "table_transformer_style": "cube",
}
query_json_penguin = SmartnoiseSynthModel.model_validate(query_json_penguin)

In [39]:
model = Synthesizer.create(
    synth=query_json_penguin.synth_name,
    epsilon=query_json_penguin.epsilon,
    **query_json_penguin.synth_params,
)
model

In [40]:
penguin_private_data = _preprocess_data(penguin_private_data, query_json_penguin)

In [41]:
penguin_transformer = _prepare_data_transformer(penguin_metadata, penguin_private_data, query_json_penguin)
penguin_transformer.transformers

{'species': <snsynth.transform.label.LabelTransformer object at 0x7f552135d8d0>, 'island': <snsynth.transform.label.LabelTransformer object at 0x7f55212c6d10>, 'sex': <snsynth.transform.label.LabelTransformer object at 0x7f55212c4650>, 'bill_length_mm': <snsynth.transform.bin.BinTransformer object at 0x7f55212c7750>, 'bill_depth_mm': <snsynth.transform.bin.BinTransformer object at 0x7f55212c7650>, 'flipper_length_mm': <snsynth.transform.bin.BinTransformer object at 0x7f55212c7bd0>, 'body_mass_g': <snsynth.transform.bin.BinTransformer object at 0x7f55212c6ad0>}


In [42]:
penguin_encoded = penguin_transformer.fit_transform(penguin_private_data, epsilon=0.0)
penguin_transformer.odometer.spent

ValueError: BinTransformer requires either epsilon or min and max.

In [43]:
model = Synthesizer.create(
    synth=query_json_penguin.synth_name,
    epsilon=query_json_penguin.epsilon,
    **query_json_penguin.synth_params,
)

In [44]:
model.fit(
    data=penguin_private_data,
    transformer=penguin_transformer,
    preprocessor_eps=1.0, #0.0,  # will error if not 0.
    nullable=query_json_penguin.nullable,
)

Spent 1.0 epsilon on preprocessor, leaving 1.0 for training


In [45]:
vars(model)

{'epsilon': 1.0,
 'delta': None,
 'reporting_length': 3,
 'percentile_percentage': 99,
 'percentile_epsilon_proportion': 0.01,
 'accuracy_mode': <AccuracyMode at 0x7f5521f1d9f0>,
 'number_of_records_epsilon_proportion': 0.005,
 'fabrication_mode': <FabricationMode at 0x7f5521f1e430>,
 'empty_value': '',
 'use_synthetic_counts': False,
 'weight_selection_percentile': 95,
 'aggregate_counts_scale_factor': None,
 'verbose': False,
 'preprocessed': False,
 'parameters': <DpAggregateSeededParameters at 0x7f55d1c23840>,
 'synth': <DpAggregateSeededSynthesizer at 0x7f5521277b80>,
 'dataset': <Dataset at 0x7f5521108030>,
 'pandas': False,
 '_transformer': <snsynth.transform.table.TableTransformer at 0x7f55212c6c10>}

In [ ]:
samples = model.sample(5)
print(samples)

## Serialise

In [45]:
import pickle
from base64 import b64encode
def serialise_model(model) -> str:
    serialised = b64encode(pickle.dumps(model))
    return serialised.decode("utf-8")

In [57]:
str_model = serialise_model(model)
str_model

In [59]:
b64_model = base64.b64decode(str_model)
b64_model

In [ ]:
new_model = pickle.loads(b64_model)
new_model

In [60]:
samples = new_model.sample(5)
print(samples)

   bill_length_mm
0       46.956818
1       45.232080
2       55.124942
3       50.191850
4       57.324367


In [63]:
import json
j_obj = json.dumps(model)

TypeError: Object of type DPCTGAN is not JSON serializable

In [46]:
import base64
import json
from base64 import b64encode
pickled_model = b64encode(pickle.dumps(model))
depickled_model = base64.b64decode(pickled_model)
json_model = json.loads(depickled_model)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [ ]:
samples = json_model.sample(100)
print(samples)